# FileToLink Bot - Deploy to Heroku

This notebook deploys the FileToLink Telegram Bot directly to Heroku from Google Colab.

**Requirements:**
- A Heroku account (free at [heroku.com](https://heroku.com))
- Heroku API Key (Account Settings > API Key)
- Telegram Bot Token (from [@BotFather](https://t.me/BotFather))
- Telegram API ID & API Hash (from [my.telegram.org](https://my.telegram.org))

---

## Step 1: Enter Your Configuration
Fill in your details below, then run all cells in order.

In [ ]:
#@title Configuration (Fill these in)
HEROKU_API_KEY = "" #@param {type:"string"}
HEROKU_APP_NAME = "" #@param {type:"string"}
HEROKU_EMAIL = "" #@param {type:"string"}
BOT_TOKEN = "" #@param {type:"string"}
API_ID = "" #@param {type:"string"}
API_HASH = "" #@param {type:"string"}
GITHUB_REPO = "https://github.com/kakarotoncloud/Filetolink" #@param {type:"string"}

# Validate inputs
assert HEROKU_API_KEY, "Please enter your Heroku API Key"
assert HEROKU_APP_NAME, "Please enter a Heroku app name"
assert HEROKU_EMAIL, "Please enter your Heroku email"
assert BOT_TOKEN, "Please enter your Telegram Bot Token"
assert API_ID, "Please enter your Telegram API ID"
assert API_HASH, "Please enter your Telegram API Hash"
print("Configuration saved!")

## Step 2: Install Heroku CLI

In [ ]:
%%bash
curl https://cli-assets.heroku.com/install.sh | sh
heroku --version

## Step 3: Login to Heroku

In [ ]:
import os
os.environ["HEROKU_API_KEY"] = HEROKU_API_KEY

# Write Heroku credentials
with open(os.path.expanduser("~/.netrc"), "w") as f:
    f.write(f"machine api.heroku.com\n")
    f.write(f"  login {HEROKU_EMAIL}\n")
    f.write(f"  password {HEROKU_API_KEY}\n")
    f.write(f"machine git.heroku.com\n")
    f.write(f"  login {HEROKU_EMAIL}\n")
    f.write(f"  password {HEROKU_API_KEY}\n")
os.chmod(os.path.expanduser("~/.netrc"), 0o600)

!heroku auth:whoami
print("Heroku login successful!")

## Step 4: Clone Repository

In [ ]:
import os
if os.path.exists("/content/Filetolink"):
    !rm -rf /content/Filetolink
!git clone {GITHUB_REPO} /content/Filetolink
%cd /content/Filetolink
!ls -la
print("\nRepository cloned successfully!")

## Step 5: Create Heroku App & Add PostgreSQL

In [ ]:
import subprocess, json

# Check if app exists, create if not
result = subprocess.run(["heroku", "apps:info", HEROKU_APP_NAME], capture_output=True, text=True)
if result.returncode != 0:
    print(f"Creating Heroku app: {HEROKU_APP_NAME}")
    !heroku create {HEROKU_APP_NAME}
else:
    print(f"App {HEROKU_APP_NAME} already exists, using it.")
    !heroku git:remote -a {HEROKU_APP_NAME}

# Add PostgreSQL addon
result = subprocess.run(["heroku", "addons", "--app", HEROKU_APP_NAME], capture_output=True, text=True)
if "postgresql" not in result.stdout.lower():
    print("Adding PostgreSQL...")
    !heroku addons:create heroku-postgresql:essential-0 --app {HEROKU_APP_NAME}
else:
    print("PostgreSQL already attached.")

print(f"\nApp URL: https://{HEROKU_APP_NAME}.herokuapp.com")

## Step 6: Set Environment Variables

In [ ]:
base_url = f"https://{HEROKU_APP_NAME}.herokuapp.com"

!heroku config:set \
  BOT_TOKEN="{BOT_TOKEN}" \
  API_ID="{API_ID}" \
  API_HASH="{API_HASH}" \
  BASE_URL="{base_url}" \
  NODE_ENV="production" \
  --app {HEROKU_APP_NAME}

print("\nEnvironment variables set!")
print(f"BASE_URL: {base_url}")

## Step 7: Deploy to Heroku

In [ ]:
%cd /content/Filetolink

# Set git config
!git config user.email "{HEROKU_EMAIL}"
!git config user.name "FileToLink Deploy"

# Add Heroku remote
!heroku git:remote -a {HEROKU_APP_NAME}

# Set Node.js buildpack
!heroku buildpacks:set heroku/nodejs --app {HEROKU_APP_NAME}

# Push to Heroku
!git push heroku main --force

print("\nDeployment complete!")

## Step 8: Initialize Database & Verify

In [ ]:
import time

# Scale the worker dyno
!heroku ps:scale web=1 --app {HEROKU_APP_NAME}

# Wait for app to start
print("Waiting for app to start...")
time.sleep(15)

# Check logs
!heroku logs --tail --num 30 --app {HEROKU_APP_NAME}

print(f"\n{'='*50}")
print(f"Deployment Complete!")
print(f"{'='*50}")
print(f"App URL:  https://{HEROKU_APP_NAME}.herokuapp.com")
print(f"Bot:      https://t.me/your_bot_username")
print(f"\nSend a file to your bot on Telegram to get started!")

---
## Useful Commands
Run these cells anytime to manage your deployment.

In [ ]:
#@title View Logs
!heroku logs --tail --num 50 --app {HEROKU_APP_NAME}

In [ ]:
#@title Restart App
!heroku restart --app {HEROKU_APP_NAME}
print("App restarted!")

In [ ]:
#@title Update Deployment (pull latest from GitHub & redeploy)
%cd /content/Filetolink
!git pull origin main
!git push heroku main --force
print("Updated and redeployed!")

In [ ]:
#@title Delete App (CAUTION)
# Uncomment the line below to delete the app
# !heroku apps:destroy {HEROKU_APP_NAME} --confirm {HEROKU_APP_NAME}